In [33]:
import numpy as np
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import mean_squared_error as mse
from skimage.io import imread
from skimage import img_as_float
from skimage.transform import resize

import lpips
import torch
from PIL import Image
from torchvision.transforms import ToTensor

def evaluate_images(original_path, super_res_path):
    # Loading images
    original = img_as_float(imread(original_path))
    super_res = img_as_float(imread(super_res_path))
    
    # resizing the sr images if needed
    if original.shape != super_res.shape:
        super_res = resize(super_res, original.shape, anti_aliasing=True)

    # Calculate PSNR, SSIM and MSE
    psnr_value = psnr(original, super_res, data_range=original.max() - original.min())
    ssim_value = ssim(original, super_res, win_size=5, data_range= 1 ,channel_axis=-1)
    mse_value = mse(original, super_res)
    
    return psnr_value, ssim_value, mse_value

def load_image_as_tensor(image_path):
    image = Image.open(image_path).convert('RGB')
    tensor = ToTensor()(image).unsqueeze(0)  # Add batch dimension
    return tensor

def evaluate_lpips(original_path, super_res_path, model = lpips.LPIPS(net='alex')):
    
    # load images as tensors
    image1 = load_image_as_tensor(original_path)
    image2 = load_image_as_tensor(super_res_path)  

    image1, image2 = image1.cuda(), image2.cuda()
    model = model.cuda()  

    with torch.no_grad():
        distance = model(image1, image2)

    return distance.item()

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


C:\Users\zhujy\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\zhujy\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Users\zhujy\AppData\Roaming\Python\Python311\site-packages\lpips\weights\v0.1\alex.pth


In [36]:
import os

original_folder = "..\DIV2K_valid_HR\DIV2K_valid_HR"
super_res_folder = "validation_output"

# loading alex model for lpips eval
lpips_model = lpips.LPIPS(net='alex')

psnr_aver, ssim_aver, mse_aver, lpips_aver = [], [], [], []

# Assuming file names are consistent between folders
for file_name in os.listdir(super_res_folder):

    # trimming the output file name
    file_name_s = file_name[:-6] + file_name[-4:]

    original_path = os.path.join(original_folder, file_name_s)
    super_res_path = os.path.join(super_res_folder, file_name)

    # Evaluate images
    psnr_value, ssim_value, mse_value = evaluate_images(original_path, super_res_path)
    lpips_value = evaluate_lpips(original_path, super_res_path)
    
    psnr_aver.append(psnr_value)
    ssim_aver.append(ssim_value)
    mse_aver.append(mse_value)
    lpips_aver.append(lpips_value)


Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


C:\Users\zhujy\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\zhujy\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Users\zhujy\AppData\Roaming\Python\Python311\site-packages\lpips\weights\v0.1\alex.pth
PSNR = 25.75511919968427, SSIM = 0.6955485732967995, MSE = 0.0033952755938017284, LPIPS = 0.17875550128519535
PSNR = 18.419705661402272, SSIM = 0.332301333047278, MSE = 0.014388960946473352, LPIPS = 0.3721003234386444


In [38]:

print(f"PSNR = {np.mean(psnr_aver)}, SSIM = {np.mean(ssim_aver)}, MSE = {np.mean(mse_aver)}, LPIPS = {np.mean(lpips_aver)}")
print(f"PSNR = {np.min(psnr_aver)}, SSIM = {np.min(ssim_aver)}, MSE = {np.max(mse_aver)}, LPIPS = {np.max(lpips_aver)}")


PSNR = 25.75511919968427, SSIM = 0.6955485732967995, MSE = 0.0033952755938017284, LPIPS = 0.17875550128519535
PSNR = 18.419705661402272, SSIM = 0.332301333047278, MSE = 0.014388960946473352, LPIPS = 0.3721003234386444
